In [1]:
%pip install numpy
%pip install pandas
%pip install scikit-learn   
%pip install matplotlib
%pip install tensorflow
%pip install langchain
%pip install langchain-community
%pip install pinecone-client
%pip install -qU langchain-pinecone pinecone-notebooks
%pip install -qU langchain-openai
%pip install -qU langchain-text-splitters
%pip install glob2
%pip install unstructured
%pip install nltk
%pip install ipython-secrets


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note

In [2]:
!git clone https://github.com/kiritbasu/Fake-Apache-Log-Generator

fatal: destination path 'Fake-Apache-Log-Generator' already exists and is not an empty directory.


In [2]:
import glob
import nltk
import unstructured
import pandas as pd
import numpy as np
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

On the Ubuntu Linux distribution, for example, access log records will be written to the following location by default:

/var/log/apache2/access.log

However since I'm using Fedora distribution and I'm not running a server, I'll synthetically generate an access log and work on it.

127.0.0.1 - Scott [10/Dec/2019:13:55:36 -0700] "GET /server-status HTTP/1.1" 200 2326

The fields in the above sample record represent the following:

    127.0.0.1 - IP address of the client that made the request;
    The hyphen defining the second field in the log file is the identity of the client. This field is often returned as a hyphen and Apache’s HTTP server documentation recommends that this particular field not be relied upon except in the case of a controlled internal network.
    Scott - userid of the person requesting the resource;
    [10/Dec/2019:13:55:36 -0700] - date and time of the request;
    “GET /server-status HTTP/1.1" - request type and resource being requested;
    200 - HTTP response status code;
    2326 - size of the object returned to the client.

This is an example of a Common Log Format which I'm going to generate the data I'll be working on.

In [3]:
loader = DirectoryLoader("Fake-Apache-Log-Generator", glob = "*.log")

In [4]:
logs = loader.load()

In [5]:
logs[0]

Document(metadata={'source': 'Fake-Apache-Log-Generator/access_log_20240820-002317.log'}, page_content='223.12.71.161 - - [20/Aug/2024:00:27:56 +0300] "PUT /posts/posts/explore HTTP/1.0" 200 5014 "http://wilson-chandler.org/tags/bloglogin.jsp" "Mozilla/5.0 (Windows NT 5.0; unm-US; rv:1.9.0.20) Gecko/7228-09-17 02:36:08.828094 Firefox/3.8"\n\n218.34.47.129 - - [20/Aug/2024:00:28:45 +0300] "GET /list HTTP/1.0" 200 4992 "https://perez-sanchez.com/category/tags/postslogin.html" "Mozilla/5.0 (X11; Linux x86_64; rv:1.9.7.20) Gecko/9287-10-06 19:19:30.603241 Firefox/3.6.8"\n\n140.132.21.231 - - [20/Aug/2024:00:30:00 +0300] "POST /explore HTTP/1.0" 200 4926 "https://schwartz.info/posts/categorypost.html" "Mozilla/5.0 (Windows 95; hy-AM; rv:1.9.1.20) Gecko/6152-12-27 20:03:29.926117 Firefox/3.8"\n\n220.165.109.138 - - [20/Aug/2024:00:32:24 +0300] "GET /list HTTP/1.0" 200 5003 "https://smith.biz/categoryfaq.htm" "Mozilla/5.0 (X11; Linux i686; rv:1.9.5.20) Gecko/3858-06-08 03:09:42.436127 Firefox

Since I'm using pinecone as a vector db which is a SaaS and OpenAI's embedding and chatgpt 3-5 models, I'm leaving my api keys here since the repo will remain private.

OPENAI: sk-1vPDBOo-6UQ-pP-6CYwAbYStCD2As8-9hezpk2Cg_YT3BlbkFJoRqGZKXz7lZoQZFELiaZqtGQ_KHtMr_KVZXXTUHiYA

Pinecone: 47f73cf2-9eee-41b3-a896-22a4ac99cb77

In [15]:
from ipython_secrets import *
import openai
import os
PINECONE_API_KEY = get_secret("PINECONE_API_KEY")
OPENAI_API_KEY = get_secret("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [16]:
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)
index_name = "pineconerag"

In [17]:
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(logs)

In [18]:
split_docs[25]

Document(metadata={'source': 'Fake-Apache-Log-Generator/access_log_20240820-002317.log'}, page_content='188.172.155.77 - - [20/Aug/2024:18:32:37 +0300] "DELETE /explore HTTP/1.0" 301 5008 "https://www.gray.com/tagabout.html" "Mozilla/5.0 (compatible; MSIE 5.0; Windows NT 5.1; Trident/4.1)"\n\n162.165.167.81 - - [20/Aug/2024:18:35:18 +0300] "GET /apps/cart.jsp?appID=7938 HTTP/1.0" 200 5006 "https://www.fletcher.com/explore/tag/listmain.htm" "Opera/9.54. (X11; Linux x86_64; nso-ZA) Presto/2.9.177 Version/12.00"\n\n131.117.10.252 - - [20/Aug/2024:18:37:21 +0300] "PUT /search/tag/list HTTP/1.0" 200 5005 "http://www.martin.biz/listindex.php" "Mozilla/5.0 (Linux; Android 4.4.3) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/47.0.862.0 Safari/535.2"\n\n70.130.247.167 - - [20/Aug/2024:18:38:13 +0300] "POST /apps/cart.jsp?appID=9936 HTTP/1.0" 200 5066 "https://www.rush.info/mainprivacy.jsp" "Mozilla/5.0 (X11; Linux x86_64; rv:1.9.7.20) Gecko/8274-10-07 12:16:23.871216 Firefox/11.0"\n\n209.189.104

In [19]:
vectorStore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name = index_name)

In [20]:
query = "GET command at 14:03:2027"
similar_docs = vectorStore.similarity_search(query)

In [22]:
similar_docs[0]

Document(metadata={'source': 'Fake-Apache-Log-Generator/access_log_20240820-002317.log'}, page_content='29.214.35.216 - - [24/Aug/2024:05:51:24 +0300] "GET /apps/cart.jsp?appID=1730 HTTP/1.0" 200 5096 "https://www.adams.info/explore/search/searchterms.htm" "Mozilla/5.0 (iPhone; CPU iPhone OS 6_1_6 like Mac OS X) AppleWebKit/533.2 (KHTML, like Gecko) CriOS/38.0.848.0 Mobile/92P402 Safari/533.2"\n\n153.60.138.160 - - [24/Aug/2024:05:55:18 +0300] "PUT /wp-content HTTP/1.0" 200 5045 "https://figueroa-reed.com/mainregister.htm" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/533.2 (KHTML, like Gecko) Chrome/41.0.836.0 Safari/533.2"\n\n89.124.142.230 - - [24/Aug/2024:05:56:06 +0300] "GET /posts/posts/explore HTTP/1.0" 200 4908 "http://www.hayes-thompson.info/app/category/searchhomepage.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 4_3_5 like Mac OS X) AppleWebKit/532.0 (KHTML, like Gecko) FxiOS/16.3y5574.0 Mobile/79T848 Safari/532.0"\n\n205.186.148.238 - - [24/Aug/2024:05:58:09 +0300] "PUT /wp-admi

In [29]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature = 0)
qa_chain = load_qa_chain(llm, chain_type="stuff")

qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorStore.as_retriever())

In [30]:
query = "Who did post a GET request at night closest to this system's timestamp?"
response = qa.run(query)

In [31]:
response

"The user with IP address 98.164.214.181 posted a GET request at night closest to the system's timestamp."